# Configuração, Inicialização e Carregamento de Dados

In [1]:
!pip install pyspark # Instala a biblioteca PySpark no ambiente de execução

In [3]:
import pyspark # Importa a biblioteca principal do PySpark
from pyspark.sql import SparkSession # Importa a classe SparkSession, o ponto de entrada para o Spark
from pyspark.sql.functions import * # Importa todas as funções de agregação, manipulação de colunas e datas, como count_distinct, approx_count_distinct, avg, col, month, year, etc.
from pyspark.sql.window import Window # Importa a Classe Window

In [4]:
from google.colab import drive # Importa o módulo drive do Google Colab
drive.mount('/content/drive') # Monta o Google Drive no ambiente

Mounted at /content/drive


In [8]:
spark = SparkSession.builder.getOrCreate() # Cria ou obtém a instância da SparkSession

In [9]:
produtos = spark.read.csv("/content/drive/MyDrive/Material de apoio - M27/produtos.csv", header=True, inferSchema=True) # Lê o DataFrame de produtos
vendedores = spark.read.csv("/content/drive/MyDrive/Material de apoio - M27/vendedores.csv", header=True, inferSchema=True) # Lê o DataFrame de vendedores
clientes = spark.read.csv("/content/drive/MyDrive/Material de apoio - M27/clientes.csv", header=True, inferSchema=True) # Lê o DataFrame de clientes
itens_pedido = spark.read.csv("/content/drive/MyDrive/Material de apoio - M27/itens_pedido.csv", header=True, inferSchema=True) # Lê o DataFrame de itens_pedido
pagamentos_pedido = spark.read.csv("/content/drive/MyDrive/Material de apoio - M27/pagamentos_pedido.csv", header=True, inferSchema=True) # Lê o DataFrame de pagamentos_pedido
avaliacoes_pedido = spark.read.csv("/content/drive/MyDrive/Material de apoio - M27/avaliacoes_pedido.csv", header=True, inferSchema=True) # Lê o DataFrame de avaliacoes_pedido
pedidos = spark.read.csv("/content/drive/MyDrive/Material de apoio - M27/pedidos.csv", header=True, inferSchema=True) # Lê o DataFrame de pedidos

# Análise Estatística (Contagens Distintas)

In [10]:
produtos.show() # Exibe as primeiras linhas de produtos

+--------------------+--------------------+--------------------+-------------------------+------------------------+--------------+----------------------+-----------------+------------------+
|          id_produto|   categoria_produto|tamanho_nome_produto|tamanho_descricao_produto|quantidade_fotos_produto|peso_produto_g|comprimento_produto_cm|altura_produto_cm|largura_produto_cm|
+--------------------+--------------------+--------------------+-------------------------+------------------------+--------------+----------------------+-----------------+------------------+
|1e9e8ef04dbcff454...|          perfumaria|                  40|                      287|                       1|           225|                    16|               10|                14|
|3aa071139cb16b67c...|               artes|                  44|                      276|                       1|          1000|                    30|               18|                20|
|96bd76ec8810374ed...|       esporte_lazer|  

In [14]:
# Agrupa por categoria_produto e calcula: 1) A contagem exata de IDs de produto únicos (count_distinct); 2) A contagem aproximada de IDs de produto únicos (approx_count_distinct), que é mais rápida
qtd_produto = produtos.groupBy('categoria_produto').agg(
    count_distinct('id_produto').alias('contagem unicos exata'),
    approx_count_distinct('id_produto').alias('contagem unicos aproximada')
)
qtd_produto.show() # Exibe os resultados comparando as duas métricas

+--------------------+---------------------+--------------------------+
|   categoria_produto|contagem unicos exata|contagem unicos aproximada|
+--------------------+---------------------+--------------------------+
|                 pcs|                   30|                        30|
|               bebes|                  919|                       873|
|               artes|                   55|                        56|
|           cine_foto|                   28|                        29|
|    moveis_decoracao|                 2657|                      2715|
|            pc_gamer|                    3|                         3|
|tablets_impressao...|                    9|                         9|
|construcao_ferram...|                  400|                       401|
|fashion_roupa_mas...|                   95|                        98|
|   artigos_de_festas|                   26|                        26|
|    artigos_de_natal|                   65|                    

# Análise Temporal e Funções de Janela

In [11]:
pedidos_completo_df = pedidos.dropna() # Remove linhas com qualquer valor nulo de pedidos

In [13]:
# Calcula a diferença de tempo entre a aprovação e a entrega, convertendo datas para long (segundos), subtraindo, e dividindo por 86400 (segundos em um dia) para obter dias
pedidos_completos_df = pedidos_completo_df.withColumn(
    "tempo_envio_dias",
    (col('data_entrega_cliente').cast("long") - col('data_aprovacao_pedido').cast("long")) / 86400
)
pedidos_completos_df.show() # Exibe o DataFrame com a nova coluna tempo_envio_dias

+--------------------+--------------------+-------------+-------------------+---------------------+-------------------------+--------------------+---------------------+------------------+
|           id_pedido|          id_cliente|status_pedido| data_compra_pedido|data_aprovacao_pedido|data_envio_transportadora|data_entrega_cliente|data_estimada_entrega|  tempo_envio_dias|
+--------------------+--------------------+-------------+-------------------+---------------------+-------------------------+--------------------+---------------------+------------------+
|e481f51cbdc54678b...|9ef432eb625129730...|    delivered|2017-10-02 10:56:33|  2017-10-02 11:07:15|      2017-10-04 19:55:00| 2017-10-10 21:25:13|  2017-10-18 00:00:00| 8.429143518518519|
|53cdb2fc8bc7dce0b...|b0830fb4747a6c6d2...|    delivered|2018-07-24 20:41:37|  2018-07-26 03:24:27|      2018-07-26 14:31:00| 2018-08-07 15:27:45|  2018-08-13 00:00:00|12.502291666666666|
|47770eb9100c2d0c4...|41ce2a54c0b03bf34...|    delivered|201

In [15]:
# Define uma partição: agrupa por status_pedido, ordena por data_aprovacao_pedido e define o frame da janela como todos os registros desde o início da partição (unboundedPreceding) até o registro atual (currentRow)
janela_acumulativa = Window.partitionBy("status_pedido").orderBy("data_aprovacao_pedido").rowsBetween(Window.unboundedPreceding, Window.currentRow)


# Aplica a função de janela, calculando a média (avg) do tempo_envio_dias para cada registro, usando a janela definida. O resultado é a média cumulativa de tempo de envio até aquele ponto na partição
pedidos_completos_df = pedidos_completos_df.withColumn(
    "media_tempo_envio_cumulativa",
    avg("tempo_envio_dias").over(janela_acumulativa)
)
pedidos_completos_df.show() # Exibe o DataFrame com a média cumulativa

+--------------------+--------------------+-------------+-------------------+---------------------+-------------------------+--------------------+---------------------+------------------+----------------------------+
|           id_pedido|          id_cliente|status_pedido| data_compra_pedido|data_aprovacao_pedido|data_envio_transportadora|data_entrega_cliente|data_estimada_entrega|  tempo_envio_dias|media_tempo_envio_cumulativa|
+--------------------+--------------------+-------------+-------------------+---------------------+-------------------------+--------------------+---------------------+------------------+----------------------------+
|65d1e226dfaeb8cdc...|70fc57eeae2926759...|     canceled|2016-10-03 21:01:41|  2016-10-04 10:18:57|      2016-10-25 12:14:28| 2016-11-08 10:58:34|  2016-11-25 00:00:00|35.027511574074076|          35.027511574074076|
|770d331c84e5b214b...|6c57e6119369185e5...|     canceled|2016-10-07 14:52:30|  2016-10-07 15:07:10|      2016-10-11 15:07:11| 2016-1

In [16]:
# Define uma partição: agrupa por status_pedido, ordena por data_aprovacao_pedido e define o frame da janela para incluir os 9 registros anteriores (-9) e o registro atual (0). Isso cria uma média móvel de 10 registros
janela_10dias = Window.partitionBy("status_pedido").orderBy("data_aprovacao_pedido").rowsBetween(-9, 0)


# Aplica a função de janela para calcular a média móvel do tempo_envio_dias dentro da janela de 10 dias.
pedidos_completos_df = pedidos_completos_df.withColumn(
    "media_tempo_envio_cumulativa",
    avg("tempo_envio_dias").over(janela_10dias)
)
pedidos_completos_df.show() # Exibe o DataFrame com a média móvel.

+--------------------+--------------------+-------------+-------------------+---------------------+-------------------------+--------------------+---------------------+------------------+----------------------------+
|           id_pedido|          id_cliente|status_pedido| data_compra_pedido|data_aprovacao_pedido|data_envio_transportadora|data_entrega_cliente|data_estimada_entrega|  tempo_envio_dias|media_tempo_envio_cumulativa|
+--------------------+--------------------+-------------+-------------------+---------------------+-------------------------+--------------------+---------------------+------------------+----------------------------+
|65d1e226dfaeb8cdc...|70fc57eeae2926759...|     canceled|2016-10-03 21:01:41|  2016-10-04 10:18:57|      2016-10-25 12:14:28| 2016-11-08 10:58:34|  2016-11-25 00:00:00|35.027511574074076|          35.027511574074076|
|770d331c84e5b214b...|6c57e6119369185e5...|     canceled|2016-10-07 14:52:30|  2016-10-07 15:07:10|      2016-10-11 15:07:11| 2016-1

#  Reiteração do Cálculo Cumulativo

In [18]:
from pyspark.sql.window import Window # Importa a classe Window novamente
from pyspark.sql.functions import col, avg

#definindo a janela de partição por status e ordenando por data de aprovação
windowSpec = Window.partitionBy("status_pedido").orderBy("data_aprovacao_pedido").rowsBetween(Window.unboundedPreceding, Window.currentRow)

pedidos_ordenados = pedidos.orderBy('data_aprovacao_pedido') # Cria um DataFrame ordenado pela data de aprovação (necessário para funções de janela sequenciais)
pedidos_ordenados = pedidos_ordenados.dropna() # Remove nulos no DataFrame ordenado (erro de sintaxe no código original)
print(pedidos.count())

# calculando a média cumulativa de tempo de envio
# Calcula a coluna tempo_envio_dias e, em seguida, calcula a média cumulativa usando a windowSpec
tempo_medio_cumulativo = pedidos_ordenados.withColumn(
    "tempo_envio_dias",
    (col('data_entrega_cliente').cast("long") -col("data_aprovacao_pedido").cast("long")) / 86400
).withColumn(
    "media_tempoenvio_cumulativa",
    avg("tempo_envio_dias").over(windowSpec)
)

tempo_medio_cumulativo.select("status_pedido", "data_aprovacao_pedido", "media_tempoenvio_cumulativa").show() # Exibe as colunas relevantes para verificar o cálculo cumulativo

99441
+-------------+---------------------+---------------------------+
|status_pedido|data_aprovacao_pedido|media_tempoenvio_cumulativa|
+-------------+---------------------+---------------------------+
|     canceled|  2016-10-04 10:18:57|         35.027511574074076|
|     canceled|  2016-10-07 15:07:10|         21.013761574074074|
|     canceled|  2016-10-09 13:36:58|         16.356400462962963|
|     canceled|  2016-10-09 14:34:30|         14.811261574074074|
|     canceled|  2016-10-10 10:40:49|         17.884150462962964|
|     canceled|  2018-02-19 20:56:05|         19.911302083333336|
|    delivered|  2016-09-15 12:16:38|          54.81319444444444|
|    delivered|  2016-10-04 09:43:32|          37.54903356481481|
|    delivered|  2016-10-04 10:19:23|         28.446720679012344|
|    delivered|  2016-10-04 10:25:46|          32.56860821759259|
|    delivered|  2016-10-04 10:28:07|         31.460384259259257|
|    delivered|  2016-10-04 10:28:19|         30.916805555555555|
|   

# Pivotamento de Dados (Tabela Dinâmica)

In [20]:
pedido_com_mes = pedidos.withColumn("mes_comra", month("data_compra_pedido")) # Extrai o mês da compra para ser usado no pivot
pedido_com_mes.show() # Exibe o DataFrame com a coluna de mês

+--------------------+--------------------+-------------+-------------------+---------------------+-------------------------+--------------------+---------------------+---------+
|           id_pedido|          id_cliente|status_pedido| data_compra_pedido|data_aprovacao_pedido|data_envio_transportadora|data_entrega_cliente|data_estimada_entrega|mes_comra|
+--------------------+--------------------+-------------+-------------------+---------------------+-------------------------+--------------------+---------------------+---------+
|e481f51cbdc54678b...|9ef432eb625129730...|    delivered|2017-10-02 10:56:33|  2017-10-02 11:07:15|      2017-10-04 19:55:00| 2017-10-10 21:25:13|  2017-10-18 00:00:00|       10|
|53cdb2fc8bc7dce0b...|b0830fb4747a6c6d2...|    delivered|2018-07-24 20:41:37|  2018-07-26 03:24:27|      2018-07-26 14:31:00| 2018-08-07 15:27:45|  2018-08-13 00:00:00|        7|
|47770eb9100c2d0c4...|41ce2a54c0b03bf34...|    delivered|2018-08-08 08:38:49|  2018-08-08 08:55:23|      

In [22]:
pedidos_com_mes = pedidos.withColumn("mes_compra", month("data_compra_pedido")) # Cria a coluna de mês corretamente (mes_compra)

# Agrupa por mes_compra, usa pivot("status_pedido") para transformar cada valor único de status_pedido em uma nova coluna, e usa agg(count("id_pedido")) para preencher os valores (contagem de pedidos)
vendas_por_mes_status = pedidos_com_mes.groupBy("mes_compra").pivot("status_pedido").agg(
    count("id_pedido").alias("total_pedidos")).orderBy("mes_compra") #


vendas_por_mes_status.show() #  Exibe o DataFrame pivotado, mostrando a contagem de pedidos por mês, segmentada por colunas de status

+----------+--------+--------+-------+---------+--------+----------+-------+-----------+
|mes_compra|approved|canceled|created|delivered|invoiced|processing|shipped|unavailable|
+----------+--------+--------+-------+---------+--------+----------+-------+-----------+
|         1|    NULL|      37|   NULL|     7819|      27|        38|     90|         58|
|         2|       1|      90|      1|     8208|      17|        38|     78|         75|
|         3|    NULL|      59|   NULL|     9549|      26|        32|    178|         49|
|         4|       1|      33|   NULL|     9101|      28|        18|    148|         14|
|         5|    NULL|      53|   NULL|    10295|      40|        29|    109|         47|
|         6|    NULL|      34|   NULL|     9234|      14|        12|     90|         28|
|         7|    NULL|      69|   NULL|    10031|      20|        12|    116|         70|
|         8|    NULL|     111|   NULL|    10544|      43|        18|     88|         39|
|         9|    NULL|

# Encerramento da Sessão

In [23]:
spark.stop() # Encerra a SparkSession e libera os recursos alocados